In [ ]:
# !pip uninstall -y torch torchaudio torchvision lightning pytorch-lightning

In [ ]:
# !pip install torch==2.4.0 torchaudio==2.4.0 torchvision==0.19.0 --index-url https://download.pytorch.org/whl/cu124
!pip install nemo_toolkit['asr']
!apt-get install portaudio19-dev
!pip install textgrid pyaudio

In [ ]:
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="nvidia/stt_ar_fastconformer_hybrid_large_pcd_v1.0")
# Define file paths
audio_file = "001001.wav"
transcription_path = "transcript.txt"
audio_dir = "audio"
output_dir = "alignment_output"
dictionary_path = "/content/arabic_msa.dict"
acoustic_model = "arabic_msa"


[NeMo I 2025-04-08 16:36:27 nemo_logging:393] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--stt_ar_fastconformer_hybrid_large_pcd_v1.0/snapshots/4591f979f911fe6f7cd858a434eac50ea386552c/stt_ar_fastconformer_hybrid_large_pcd_v1.0.nemo.


# Tasmy3

In [ ]:
import json

with open("/content/Quran.json", "r", encoding="utf-8") as f:
    quran_data = json.load(f)

    quran_texts = [((s["surah_number"], s["verse_number"],s["page"]), s["text"]) for s in quran_data]
quran_texts[:9]

[((1, 1, 1), 'بسم الله الرحمن الرحيم'),
 ((1, 2, 1), 'الحمد لله رب العالمين'),
 ((1, 3, 1), 'الرحمن الرحيم'),
 ((1, 4, 1), 'مالك يوم الدين'),
 ((1, 5, 1), 'إياك نعبد وإياك نستعين'),
 ((1, 6, 1), 'اهدنا الصراط المستقيم'),
 ((1, 7, 1), 'صراط الذين أنعمت عليهم غير المغضوب عليهم ولا الضالين'),
 ((2, 1, 2), 'الم'),
 ((2, 2, 2), 'ذلك الكتاب لا ريب فيه هدى للمتقين')]

In [ ]:
from difflib import SequenceMatcher

def find_best_match(transcription, quran_texts):
    best_match = None
    best_ratio = 0

    for (surah_ayah_page, text) in quran_texts:
        similarity = SequenceMatcher(None, transcription, text).ratio()
        if similarity > best_ratio:
            best_ratio = similarity
            best_match = (surah_ayah_page, text, similarity)

    return best_match if best_ratio > 0.6 else None


In [ ]:
from omegaconf import open_dict

decoding_cfg = asr_model.cfg.decoding
with open_dict(decoding_cfg):
    decoding_cfg.preserve_alignments = True
    decoding_cfg.compute_timestamps = True
asr_model.change_decoding_strategy(decoding_cfg)
print(asr_model.cfg.decoding) ## If it contains preserve_alignments=True or timestamp-related fields, it likely supports alignment.

In [ ]:
!pip install numpy==1.24.4

In [ ]:
audio_file_path ='/content/001001.wav'
transcriptions = asr_model.transcribe([audio_file_path], channel_selector=0)[0]

best_match = find_best_match(transcriptions.text, quran_texts)

if best_match:
    (surah_number, ayah_number,page_number), matched_text, similarity = best_match
    print(f"✅ أنت الآن عند ({surah_number}:{ayah_number}) -(صفحة {page_number}) - {matched_text} (دقة {similarity*100:.2f}%)")

    next_ayah = next((
        ((sura, aya, page), text)
        for (sura, aya, page), text in quran_texts
        if sura == surah_number and aya == ayah_number + 1
    ), None)

    if next_ayah:
        (next_sura, next_ayah_num, next_page), next_text = next_ayah
        print(f"➡️ الآية المتوقعة بعد كده: ({next_sura}:{next_ayah_num}) - {next_text} (صفحة {next_page})")
    else:
        # البحث عن أول آية في السورة التالية
      next_surah_ayah = next((
          ((sura, aya, page), text)
          for (sura, aya, page), text in quran_texts
          if sura == surah_number + 1 and aya == 1
      ), None)

      if next_surah_ayah:
          (next_sura, next_ayah_num, next_page), next_text = next_surah_ayah
          print(f"⏭️ تم الانتقال إلى السورة التالية ({next_sura}:1) - {next_text} (صفحة {next_page})")
      else:
          print("⏸️ لا توجد آيات تالية، انتهى المصحف.")



[NeMo I 2025-04-08 16:37:45 nemo_logging:393] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


Transcribing: 100%|██████████| 1/1 [00:22<00:00, 22.22s/it]


✅ أنت الآن عند (1:1) -(صفحة 1) - بسم الله الرحمن الرحيم (دقة 100.00%)
➡️ الآية المتوقعة بعد كده: (1:2) - الحمد لله رب العالمين (صفحة 1)


In [ ]:
import nemo.collections.asr as nemo_asr
import numpy as np
from difflib import SequenceMatcher
import librosa


audio_file_path = '/content/001007.wav'

# Get the actual duration of the audio file
audio_duration, sample_rate = librosa.get_duration(filename=audio_file_path), librosa.get_samplerate(audio_file_path)
print(f"Audio duration: {audio_duration:.2f}s | Sample rate: {sample_rate}Hz")
# asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name="nvidia/stt_ar_fastconformer_hybrid_large_pcd_v1.0")


# Transcribe the audio
# transcription = asr_model.transcribe([audio_file_path], channel_selector=0, return_hypotheses=True,timestamps= True)[0]
# transcription


Audio duration: 13.19s | Sample rate: 44100Hz


In [ ]:
# import nemo_asr and instantiate asr_model as above
import nemo.collections.asr as nemo_asr
asr_model = nemo_asr.models.ASRModel.from_pretrained("nvidia/stt_ar_fastconformer_hybrid_large_pcd_v1.0")

# specify flag `timestamps=True`
hypotheses = asr_model.transcribe([audio_file_path], channel_selector=0, timestamps= True)



# segment level timestamps (if model supports Punctuation and Capitalization, segment level timestamps are displayed based on punctuation otherwise complete transcription is considered as a single segment)

In [ ]:
# by default, timestamps are enabled for char, word and segment level
word_timestamps = hypotheses[0].timestamp['word'] # word level timestamps for first sample
segment_timestamps = hypotheses[0].timestamp['segment'] # segment level timestamps
char_timestamps = hypotheses[0].timestamp['char'] # char level timestamps
for word in word_timestamps:
    print(f"start: {word['start']}s - end: {word['end']}s word: {word['word']}")

start: 0.16s - end: 0.8s word: صراط
start: 1.04s - end: 1.12s word: الذين
start: 1.68s - end: 2.4s word: أنعمت
start: 2.8000000000000003s - end: 3.36s word: عليهم
start: 3.68s - end: 3.7600000000000002s word: غير
start: 4.4s - end: 5.28s word: المغضوب
start: 5.6000000000000005s - end: 6.08s word: عليهم
start: 6.4s - end: 6.48s word: ولا
start: 6.96s - end: 10.32s word: الضالين


In [ ]:


# Get time stride from model config
time_stride = asr_model.cfg.preprocessor.window_size  # e.g., 0.025s (25ms)
print(f"Time stride: {time_stride}s")

# Extract raw timestamps (frame indices) and total length
raw_timestamps = transcription.timestamp.cpu().tolist()  # Frame indices: [2, 5, 9, ...]
total_frames = transcription.length.item()  # Total frames: 165
print(f"Raw timestamps (frames): {raw_timestamps[:5]}... (total: {len(raw_timestamps)})")
print(f"Total frames: {total_frames}")

# Convert raw timestamps to seconds (before scaling)
timestamps = [t * time_stride for t in raw_timestamps]
print(f"Max timestamp (before scaling): {timestamps[-1]:.2f}s")

# Scale timestamps to match the full audio duration
if timestamps[-1] < audio_duration:
    print(f"⚠️ Model timestamps ({timestamps[-1]:.2f}s) don’t span full audio ({audio_duration:.2f}s). Scaling based on total frames.")
    scale_factor = audio_duration / (total_frames * time_stride)  # Scale based on total frames (165 * 0.025 = 4.125s)
    timestamps = [t * time_stride * scale_factor for t in raw_timestamps]
    print(f"Scaled max timestamp: {timestamps[-1]:.2f}s")

# Get the transcription text and split into words
text = transcription.text
words = text.split()
print(f"Transcription: {text}")

# Debugging: Check lengths
print(f"Timestamps: {len(timestamps)} | Words: {len(words)}")

# Map timestamps to words
word_timestamps = []
if len(timestamps) > len(words):
    print("⚠️ More timestamps than words. Grouping timestamps to approximate word boundaries.")
    timestamps_per_word = len(timestamps) // len(words)
    for i, word in enumerate(words):
        start_idx = i * timestamps_per_word
        end_idx = (i + 1) * timestamps_per_word if (i + 1) * timestamps_per_word < len(timestamps) else len(timestamps) - 1
        start_time = timestamps[start_idx]
        end_time = timestamps[end_idx]
        word_timestamps.append({"word": word, "start": start_time, "end": end_time})
else:
    print("⚠️ Fewer or equal timestamps to words. Distributing evenly over full duration.")
    step = audio_duration / len(words)
    word_timestamps = [{"word": w, "start": i * step, "end": (i + 1) * step} for i, w in enumerate(words)]

# Print the timestamps
print("🕒 تفاصيل الكلمات:")
for entry in word_timestamps:
    duration = entry["end"] - entry["start"]
    print(f"📌 كلمة: {entry['word']} | ⏳ Start: {entry['start']:.2f}s | 🏁 End: {entry['end']:.2f}s | ⏱ Duration: {duration:.2f}s")

# Stream

In [ ]:
import wave
import queue
import threading
import time

# تحميل ملف الصوت
audio_file_path = "010010.wav"
wf = wave.open(audio_file_path, 'rb')

audio_queue = queue.Queue()

def fake_audio_stream():
    while True:
        data = wf.readframes(1024)
        if not data:
            break
        audio_queue.put(data)
        time.sleep(1024 / 16000)

def live_transcribe():
    print("🎙️ استمع الآن... (محاكاة تسجيل لايف)")

    while True:
        frames = []
        while not audio_queue.empty():
            frames.append(audio_queue.get())


        if frames:
            audio_data = b''.join(frames)

            transcription = asr_model.transcribe([audio_data], channel_selector=0)[0]
            print(f"📝 النص: {transcription}")

            best_match = find_best_match(transcription, quran_texts)
            if best_match:
                (surah_number, ayah_number,page_number), matched_text, similarity = best_match
                print(f"✅ أنت الآن عند ({surah_number}:{ayah_number}) صفحة {page_number} - {matched_text} (دقة {similarity*100:.2f}%)")

                next_ayah = next(((sura, aya,page), text) for (sura, aya,page), text in quran_texts if sura == surah_number and aya == ayah_number + 1)
                print(f"➡️ الآية المتوقعة بعد كده: {next_ayah[1]}")
            else:
                print("❌ لم أتعرف على الآية")

# تشغيل المحاكاة في Thread منفصل
threading.Thread(target=fake_audio_stream, daemon=True).start()
threading.Thread(target=live_transcribe, daemon=True).start()


In [ ]:
#https://wtq-recitations.s3.us-east-2.amazonaws.com/Word_by-Word/2/002007001.mp3
#https://wtq-recitations.s3.us-east-2.amazonaws.com/Word_by-Word/[surah number]/[surah number][ayah number][word number].mp3
import os
import requests
# make a code that will download all the audio files from the above link
def download_word_by_word(surah_number, ayah_number, word_number):
    file_name = f'{surah_number:03}{ayah_number:03}{word_number:03}.mp3'
    url =f'https://wtq-recitations.s3.us-east-2.amazonaws.com/Word_by-Word/{surah_number}/{surah_number:03}{ayah_number:03}{word_number:03}.mp3'
    base_folder = 'word_by_word'
    surah_folder = f'tajweed/surah_{surah_number}'

    if not os.path.exists(base_folder):
        os.makedirs(base_folder)

    if not os.path.exists(os.path.join(base_folder, surah_folder)):
        os.makedirs(os.path.join(base_folder, surah_folder))

    file_path = os.path.join(base_folder, surah_folder, file_name)

    if os.path.exists(file_path):
        print(f'{file_name} already exists')
        return

    response = requests.get(url)
    print(str(response.status_code) + " Connection Okay: " + url)
    if response.status_code == 200:
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f'Downloaded and saved to {file_path}')
    else:
        print(f'Failed to download {file_name}')

# Production

In [ ]:
import json
from difflib import SequenceMatcher
import nemo.collections.asr as nemo_asr

class QuranASR:
    def __init__(self, model_name="nvidia/stt_ar_fastconformer_hybrid_large_pcd_v1.0", quran_json_path="/content/quran.json"):
        self.asr_model = nemo_asr.models.EncDecHybridRNNTCTCBPEModel.from_pretrained(model_name=model_name)
        self.quran_texts = self._load_quran(quran_json_path)

    def _load_quran(self, json_path):
        with open(json_path, "r", encoding="utf-8") as f:
            quran_data = json.load(f)
        return [((s["surah_number"], s["verse_number"], s["page"]), s["text"]) for s in quran_data]

    def _word_similarity(self, word1, word2):
        return SequenceMatcher(None, word1, word2).ratio()

    def _sentence_similarity(self, partial_text, full_text):
        partial_words = list(set(partial_text.split()))  # إزالة التكرار
        full_words = full_text.split()

        match_count, total_weight = 0, 0
        for p_word in partial_words:
            best_match = max((self._word_similarity(p_word, f_word) for f_word in full_words), default=0)
            if best_match > 0.7:  # رفع العتبة لتحسين الدقة
                match_count += 1
                total_weight += best_match

        return total_weight / len(partial_words) if partial_words else 0

    def _find_best_match(self, spoken_text, page_number):
        best_match, best_ratio = None, 0
        filtered_quran_texts = [entry for entry in self.quran_texts if entry[0][2] == page_number]

        for (surah_ayah_page, text) in filtered_quran_texts:
            similarity = self._sentence_similarity(spoken_text, text)
            if similarity > best_ratio:
                best_ratio = similarity
                best_match = (surah_ayah_page, text, similarity)

        return best_match if best_ratio > 0.5 else None  # رفع العتبة إلى 50% للتحسين

    def process_audio(self, audio_file_path):
        transcriptions = self.asr_model.transcribe([audio_file_path], channel_selector=0)[0]
        return transcriptions.text.strip()

    def get_next_quran_word(self, audio_file_path, page_number):
        spoken_text = self.process_audio(audio_file_path)
        best_match = self._find_best_match(spoken_text, page_number)

        if not best_match:
            return {"spoken_text": spoken_text, "error": "No matching verse found"}

        (surah_number, ayah_number, page_number), matched_text, similarity = best_match
        words = matched_text.split()
        spoken_words = list(set(spoken_text.split()))  # إزالة الكلمات المتكررة

        next_word = None
        for i in range(len(spoken_words) - 1, -1, -1):
            if spoken_words[i] in words:
                index = words.index(spoken_words[i])
                if index + 1 < len(words):
                    next_word = words[index + 1]
                break

        if not next_word:
            next_ayah = next(
                (((sura, aya, page), text) for (sura, aya, page), text in self.quran_texts if sura == surah_number and aya == ayah_number + 1), None
            )
            next_word = next_ayah[1].split()[0] if next_ayah else None

        return {
            "spoken_text": spoken_text,
            "current": {
                "surah": surah_number,
                "ayah": ayah_number,
                "page": page_number,
                "text": matched_text,
                "accuracy": round(similarity * 100, 2)
            },
            "next_word": next_word
        }


In [ ]:
quran_asr = QuranASR()

In [ ]:
results = []
for surah in os.listdir('/content/word_by_word/tajweed'):
    ayahs = os.path.join('/content/word_by_word/tajweed', surah)
    print(ayahs[-1])
    for ayah in os.listdir(ayahs):
        audio_file_path = os.path.join(ayahs, ayah)
        result = quran_asr.get_next_quran_word(audio_file_path,int(ayahs[-1]))
        results.append(result)



In [ ]:
print(json.dumps(results, ensure_ascii=False, indent=4))
with open('output.json', 'w', encoding='utf-8') as f:
       f.write(json.dumps(results, ensure_ascii=False, indent=4))

In [ ]:
import json
with open('ayahs.json', 'r', encoding='utf-8') as ayahs:
    ayahs = json.load(ayahs)
    for ayah in ayahs:
        break
        for index , word in enumerate(ayah['words']):
            download_word_by_word(ayah['surah_number'],ayah['verse_number'], index+1)


In [ ]:
!pip install PyArabic

In [ ]:
from difflib import SequenceMatcher

def word_similarity(word1, word2):
    return SequenceMatcher(None, word1, word2).ratio()

def sentence_similarity(partial_text, full_text):
    partial_words = partial_text.split()
    full_words = full_text.split()

    match_count = 0
    total_weight = 0

    for p_word in partial_words:
        best_match = max((word_similarity(p_word, f_word) for f_word in full_words), default=0)
        if best_match > 0.5:
            match_count += 1
            total_weight += best_match

    return total_weight / len(partial_words) if partial_words else 0

def _find_best_match(partial_text, page_number):
    best_match = None
    best_ratio = 0

    filtered_quran_texts = [entry for entry in quran_texts if entry[0][2] == page_number]
    print(filtered_quran_texts)
    for (surah_ayah_page, text) in filtered_quran_texts:
        similarity = sentence_similarity(partial_text, text)
        if similarity > best_ratio:
            best_ratio = similarity
            best_match = (surah_ayah_page, text, similarity)

    return best_match if best_ratio > 0.3 else None

# ✅ تجربة البحث بجزء بسيط من الآية
print(_find_best_match('بسم', 1))
print(_find_best_match('الرحمن الرحيم', 1))
print(_find_best_match('اهدنا الصراط المستقيم', 1))
